In [ ]:
import os, sys
root_dir = '../'
sys.path.append(root_dir)
from config import config
from SAM.train import load_eval
from SAM import datasets, utils

In [ ]:
dir_paths = os.path.join(root_dir, 'output')

V_dict = {}
V_pred_dict = {}
V_pdc_dict = {}
n_max = 50

for dir_path in os.listdir(dir_paths):
    print(dir_path)
    dir_path = os.path.join(root_dir, 'output', dir_path)
    path_param = os.path.join(dir_path, 'params/config.yml')
    cfg = config.load_config(path_param)
    config.check_device_config(cfg)

    cfg.data.n_all_per_dim = cfg.data.n_sam_per_dim
    config.check_config(cfg, save_cfg=False, check_path=False)
    dataset = datasets.get_dataset(cfg)
    potential_fn = utils.get_potential_fn(cfg)

    cfg.data.n_all_per_dim = n_max
    config.check_config(cfg, save_cfg=False, check_path=False)
    dataset_pdc = datasets.get_pdc_dataset(cfg)
    
    V_pred = load_eval(cfg, dir_path).item()
    V = potential_fn(dataset.x_all, dataset.mean).item()
    V_pdc = potential_fn(dataset_pdc.x_all, dataset_pdc.mean).item()

    params = (cfg.model.k_near, cfg.data.n_sam_per_dim)
    V_dict[params] = V
    V_pred_dict[params] = V_pred
    V_pdc_dict[params] = V_pdc

In [ ]:
from matplotlib import pyplot as plt

fig, axes = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True)

k_list = sorted(set(param[0] for param in V_pred_dict.keys()))
n_list = sorted(set(param[1] for param in V_pred_dict.keys()))
for k in k_list:
    V_pred_list = [V_pred_dict[(k, n)] for n in n_list]
    V_list = [V_dict[(k, n)] for n in n_list]
    V_pdc_list = [V_pdc_dict[(k, n)] for n in n_list]


    axes[0].plot(n_list, V_list, marker='o', label=f" nearest neighbors atoms")
    axes[1].plot(n_list, V_pdc_list, marker='o', label=f" nearest neighbors atoms")
    axes[2].plot(n_list, V_pred_list, marker='o', label=f" nearest neighbors atoms")